# Preprocessing SODA and Training GNNs

(Simplified, without Comments)

by Ding

For exploratory steps and comments, please see [this notebook](https://github.com/ding05/GNN_CNN_MHW_Forecasting_EEs/blob/main/preprocessing_c.ipynb).

In [1]:
!pip install geopandas

import numpy as np
from netCDF4 import Dataset
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point

     |████████████████████████████████| 1.0 MB 4.9 MB/s 
     |████████████████████████████████| 6.3 MB 38.6 MB/s 
     |████████████████████████████████| 15.4 MB 38.7 MB/s 


In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
%%bash
cp -a '/gdrive/MyDrive/soda_331_pt_l5.nc' '/content/'
cp -a '/gdrive/MyDrive/sst_anomaly.nc' '/content/'

In [4]:
soda = xr.open_dataset('soda_331_pt_l5.nc', decode_times=False)
soda

<xarray.Dataset>
Dimensions:    (depth: 1, latitude: 330, longitude: 720, time: 432)
Coordinates:
  * time       (time) float64 3.168e+08 3.195e+08 ... 1.448e+09 1.45e+09
  * depth      (depth) float32 5.034
  * latitude   (latitude) float32 -74.75 -74.25 -73.75 ... 88.75 89.25 89.75
  * longitude  (longitude) float32 0.25 0.75 1.25 1.75 ... 358.8 359.2 359.8
Data variables:
    temp       (time, depth, latitude, longitude) float32 ...
Attributes: (12/47)
    _CoordSysBuilder:              ucar.nc2.internal.dataset.conv.DefaultConv...
    acknowledgement:               The SODA3 project is funded by the US Nati...
    cdm_data_type:                 Grid
    Conventions:                   COARDS, CF-1.6, ACDD-1.3
    Created_by:                    Gennady Chepurin and Ligang Chen
    creation_date:                 October, 2016
    ...                            ...
    summary:                       SODA3.3.1 ocean state, forced by MERRA2. T...
    time_coverage_end:             2015-12-16T00:00:00Z
    time_coverage_start:           1980-01-16T00:00:00Z
    title:                         SODA 3.3.1 Ocean State, 1/2°, Global, 1980...
    Website:                       https://www.atmos.umd.edu/~ocean/
    Westernmost_Easting:           0.25

In [55]:
soda_array = soda.to_array(dim='temp')
soda_smaller = soda_array[:,:,:,::5,::5].to_dataset(dim="temp")
soda_smaller

<xarray.Dataset>
Dimensions:    (depth: 1, latitude: 66, longitude: 144, time: 432)
Coordinates:
  * time       (time) float64 3.168e+08 3.195e+08 ... 1.448e+09 1.45e+09
  * depth      (depth) float32 5.034
  * latitude   (latitude) float32 -74.75 -72.25 -69.75 ... 82.75 85.25 87.75
  * longitude  (longitude) float32 0.25 2.75 5.25 7.75 ... 352.8 355.2 357.8
Data variables:
    temp       (time, depth, latitude, longitude) float32 nan nan ... -1.636
Attributes: (12/47)
    _CoordSysBuilder:              ucar.nc2.internal.dataset.conv.DefaultConv...
    acknowledgement:               The SODA3 project is funded by the US Nati...
    cdm_data_type:                 Grid
    Conventions:                   COARDS, CF-1.6, ACDD-1.3
    Created_by:                    Gennady Chepurin and Ligang Chen
    creation_date:                 October, 2016
    ...                            ...
    summary:                       SODA3.3.1 ocean state, forced by MERRA2. T...
    time_coverage_end:             2015-12-16T00:00:00Z
    time_coverage_start:           1980-01-16T00:00:00Z
    title:                         SODA 3.3.1 Ocean State, 1/2°, Global, 1980...
    Website:                       https://www.atmos.umd.edu/~ocean/
    Westernmost_Easting:           0.25

In [66]:
start_year = 1980
end_year = 2016

In [70]:
start_month = (start_year - 1980) * 12
end_month = (end_year - 1980) * 12

soda_sst = np.zeros((end_month-start_month,1,66,144))
soda_sst[:,:,:,:] = soda_smaller.variables['temp'][0:end_month-start_month,:,:,:]

In [71]:
soda_sst = np.squeeze(soda_sst, axis=1)

soda_sst_list = soda_sst.tolist()

months = list(range(0, 432))
monthly_average_all = []

for i in range(12):
  individual_month = months[i + start_month : end_month : 12]
  average = np.zeros((66,144))
  for j in range(len(individual_month)):
    average += soda_sst[individual_month[j]]
    # average_map += np.array(individual_month[j])
  monthly_average = average / len(individual_month)
  monthly_average_all.append(monthly_average)
  print("Month " + str(i+1) + " is appended.")

Month 1 is appended.
Month 2 is appended.
Month 3 is appended.
Month 4 is appended.
Month 5 is appended.
Month 6 is appended.
Month 7 is appended.
Month 8 is appended.
Month 9 is appended.
Month 10 is appended.
Month 11 is appended.
Month 12 is appended.


In [72]:
monthly_average_all_432 = []
monthly_average_all_432 = monthly_average_all
print(len(monthly_average_all))
print(len(monthly_average_all_432))

for i in range(432 - 12):
  monthly_average_all_432.append(monthly_average_all_432[i])

print(len(monthly_average_all_432))

soda_sst_anomaly_list = []

for i in range(432):
  soda_sst_anomaly_list.append(soda_sst[i] - monthly_average_all_432[i])

12
12
432


In [73]:
soda_sst_anomaly = np.array(soda_sst_anomaly_list)

soda_sst_anomaly.shape

(432, 66, 144)

In [74]:
soda_sst_anomaly_transposed = soda_sst_anomaly.transpose(1,2,0)
soda_sst_anomaly_flattened = soda_sst_anomaly_transposed.reshape(soda_sst_anomaly.shape[1] * soda_sst_anomaly.shape[2],432)
soda_sst_anomaly_flattened.shape

(9504, 432)

In [77]:
def dropna(arr, *args, **kwarg):
    assert isinstance(arr, np.ndarray)
    dropped=pd.DataFrame(arr).dropna(*args, **kwarg).values
    if arr.ndim==1:
        dropped=dropped.flatten()
    return dropped

soda_sst_anomaly_ocean_flattened = dropna(soda_sst_anomaly_flattened)
soda_sst_anomaly_ocean_flattened.shape

(6924, 432)

In [92]:
feature_matrix = soda_sst_anomaly_ocean_flattened

In [80]:
lons, lats = np.meshgrid(soda_smaller.longitude.values, soda_smaller.latitude.values)

soda_time_1 = soda_smaller.temp.isel(depth=0,time=240)

soda_time_1_lons, soda_time_1_lats = np.meshgrid(soda_time_1.longitude.values, soda_time_1.latitude.values)

soda_masked = soda_time_1.where(abs(soda_time_1_lons) + abs(soda_time_1_lats) > 0)
soda_masked

<xarray.DataArray 'temp' (latitude: 66, longitude: 144)>
array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [-1.1247305, -1.4289024, -1.4716144, ..., -1.7336361, -1.7187185,
        -1.6495545],
       ...,
       [-1.7470648, -1.7409257, -1.7132384, ..., -1.7103977, -1.7174913,
        -1.7176341],
       [-1.7337334, -1.7335713, -1.740988 , ..., -1.7035117, -1.7163125,
        -1.7244432],
       [-1.7108924, -1.7107269, -1.7117507, ..., -1.7120712, -1.7136484,
        -1.7127907]], dtype=float32)
Coordinates:
    time       float64 9.48e+08
    depth      float32 5.034
  * latitude   (latitude) float32 -74.75 -72.25 -69.75 ... 82.75 85.25 87.75
  * longitude  (longitude) float32 0.25 2.75 5.25 7.75 ... 352.8 355.2 357.8

In [82]:
soda_masked.values.flatten()[soda_masked.notnull().values.flatten()]

len(soda_masked.values.flatten()[soda_masked.notnull().values.flatten()])

6924

In [83]:
print(soda_time_1_lons.flatten()[soda_masked.notnull().values.flatten()])
print(soda_time_1_lats.flatten()[soda_masked.notnull().values.flatten()])

[162.75 165.25 167.75 ... 352.75 355.25 357.75]
[-74.75 -74.75 -74.75 ...  87.75  87.75  87.75]


In [86]:
from sklearn.metrics.pairwise import haversine_distances

lons_ocean = soda_time_1_lons.flatten()[soda_masked.notnull().values.flatten()]
lons_ocean = lons_ocean[::]

lats_ocean = soda_time_1_lats.flatten()[soda_masked.notnull().values.flatten()]
lats_ocean = lats_ocean[::]

lons_ocean *= np.pi/180
lats_ocean *= np.pi/180

points_ocean = np.concatenate([np.expand_dims(lats_ocean.flatten(),-1), np.expand_dims(lons_ocean.flatten(),-1)],-1)

distance_ocean = 6371*haversine_distances(points_ocean)

In [87]:
distance_ocean_diag = distance_ocean
distance_ocean_diag[distance_ocean_diag==0] = 1

distance_ocean_recip = np.reciprocal(distance_ocean_diag)

distance_ocean_recip.shape

(6924, 6924)

In [88]:
adjacency_matrix = distance_ocean_recip

In [94]:
lead_month = 0

feature_matrix = feature_matrix[:,:len(feature_matrix[0])-lead_month:]
adjacency_matrix = adjacency_matrix

In [96]:
import os
import json
import math
import numpy as np
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [99]:
node_feats = torch.tensor(np.expand_dims(feature_matrix, axis=0)).float()
adj_matrix = torch.tensor(np.expand_dims(adjacency_matrix, axis=0)).float()

print(node_feats.shape)
print(adj_matrix.shape)

torch.Size([1, 6924, 432])
torch.Size([1, 6924, 6924])


In [100]:
class GCNLayer(nn.Module):

    def __init__(self, c_in, c_out):
        super().__init__()
        self.projection = nn.Linear(c_in, c_out)

    def forward(self, node_feats, adj_matrix):
        """
        Inputs:
            node_feats - Tensor with node features of shape [batch_size, num_nodes, c_in]
            adj_matrix - Batch of adjacency matrices of the graph. If there is an edge from i to j, adj_matrix[b,i,j]=1 else 0.
                         Supports directed edges by non-symmetric matrices. Assumes to already have added the identity connections.
                         Shape: [batch_size, num_nodes, num_nodes]
        """
        # Num neighbours = number of incoming edges
        num_neighbours = adj_matrix.sum(dim=-1, keepdims=True)
        node_feats = self.projection(node_feats)
        node_feats = torch.bmm(adj_matrix, node_feats)
        node_feats = node_feats / num_neighbours
        return node_feats

In [101]:
print("Node features:\n", node_feats)
print("\nAdjacency matrix:\n", adj_matrix)

Node features:
 tensor([[[-7.9552e-02, -7.8261e-02,  4.1625e-02,  ..., -3.3746e-03,
           8.1046e-03, -4.8462e-02],
         [ 1.1985e+00,  3.5482e-01,  1.2477e-01,  ..., -1.4804e-02,
          -9.4055e-03, -1.1167e-01],
         [ 1.7171e+00,  1.1768e+00,  1.3464e-01,  ..., -1.2377e-04,
           1.9625e-03, -8.1020e-02],
         ...,
         [-9.9387e-02, -9.5504e-02, -9.8592e-02,  ...,  8.6997e-02,
           6.8804e-02,  6.4023e-02],
         [-1.0047e-01, -9.3609e-02, -9.5575e-02,  ...,  8.7748e-02,
           7.1659e-02,  6.7239e-02],
         [-9.4762e-02, -8.9556e-02, -9.3895e-02,  ...,  8.8551e-02,
           7.4358e-02,  7.0678e-02]]])

Adjacency matrix:
 tensor([[[1.0000e+00, 1.3677e-02, 6.8402e-03,  ..., 5.3864e-05,
          5.3872e-05, 5.3880e-05],
         [1.3677e-02, 1.0000e+00, 1.3677e-02,  ..., 5.3859e-05,
          5.3864e-05, 5.3872e-05],
         [6.8402e-03, 1.3677e-02, 1.0000e+00,  ..., 5.3855e-05,
          5.3859e-05, 5.3864e-05],
         ...,
       

In [102]:
temp_list = []
for i in range(0, feature_matrix.shape[1]):
  element = [0] * feature_matrix.shape[1]
  element[i] = 1
  element = [float(item) for item in element] # Convert the type into float.
  temp_list.append(element)

layer = GCNLayer(c_in=feature_matrix.shape[1], c_out=feature_matrix.shape[1])
layer.projection.weight.data = torch.tensor(temp_list)

layer.projection.bias.data = torch.Tensor([0] * feature_matrix.shape[1])

In [109]:
with torch.no_grad():
    out_feats = layer(node_feats, adj_matrix)

print("Adjacency matrix")
print(np.round(np.array(adj_matrix), decimals=3))
print("Input features")
print(np.round(np.array(node_feats), decimals=3))
print("Output features")
print(np.round(np.array(out_feats), decimals=3))

Adjacency matrix
[[[1.    0.014 0.007 ... 0.    0.    0.   ]
  [0.014 1.    0.014 ... 0.    0.    0.   ]
  [0.007 0.014 1.    ... 0.    0.    0.   ]
  ...
  [0.    0.    0.    ... 1.    0.092 0.046]
  [0.    0.    0.    ... 0.092 1.    0.092]
  [0.    0.    0.    ... 0.046 0.092 1.   ]]]
Input features
[[[-0.08  -0.078  0.042 ... -0.003  0.008 -0.048]
  [ 1.198  0.355  0.125 ... -0.015 -0.009 -0.112]
  [ 1.717  1.177  0.135 ... -0.     0.002 -0.081]
  ...
  [-0.099 -0.096 -0.099 ...  0.087  0.069  0.064]
  [-0.1   -0.094 -0.096 ...  0.088  0.072  0.067]
  [-0.095 -0.09  -0.094 ...  0.089  0.074  0.071]]]
Output features
[[[ 0.296  0.161  0.13  ...  0.097  0.094  0.011]
  [ 0.9    0.37   0.17  ...  0.091  0.085 -0.019]
  [ 1.143  0.759  0.176 ...  0.097  0.09  -0.006]
  ...
  [-0.085 -0.065 -0.066 ...  0.17   0.134  0.142]
  [-0.085 -0.065 -0.065 ...  0.17   0.135  0.143]
  [-0.084 -0.064 -0.065 ...  0.171  0.136  0.144]]]
